In [70]:
import re
from pathlib import Path
from dataclasses import dataclass
import pandas as pd
import numpy as np
from typing import List, Dict


In [ ]:

#INGESTA RAW DATA
#---------------------------------------------------------------------
#RUTA
folder = Path(r"C:\Users\MI PC\Desktop\Challenge Data Science\CSV")

#ABRIR TODOS LOS ARCHIVOS JUNTOS
files = list(folder.glob("*.xlsx"))
print(files)   # para verificar

# leer y concatenar todos los excels
dfs = []
for f in files:
    df = pd.read_excel(f)
    df["Sucursal"] = f.stem   
    dfs.append(df)

Data_raw = pd.concat(dfs, ignore_index=True)
print(Data_raw.shape)
Data_raw.head()


[WindowsPath('C:/Users/MI PC/Desktop/Challenge Data Science/CSV/Ventas Alvisa.xlsx'), WindowsPath('C:/Users/MI PC/Desktop/Challenge Data Science/CSV/Ventas CiudadSF.xlsx'), WindowsPath('C:/Users/MI PC/Desktop/Challenge Data Science/CSV/Ventas LaHorqueta.xlsx'), WindowsPath('C:/Users/MI PC/Desktop/Challenge Data Science/CSV/Ventas VicenteLopez.xlsx')]
(123718, 24)


,FECHA,NUMERO_TICKET,CANTIDAD_TICKET,ID_SUCURSAL,DESCRIP_SUCURSAL,ID_ZONA_SUPERVISION,DESC_ZONA_SUPERVICION,ID_ARTICULO,DESC_ARTICULO,FAMILIA,...,DESC_RUBRO,SUBRUBRO,DESC_SUBRUBRO,CANTIDAD_VENDIDA,VALOR_ARTICULO,VENTA_BRUTA,MONTO_IMPUESTOS_INTERNOS,MONTO_IVA,COSTO_ARTICULO,Sucursal
0,2024-01-05 10:58:49.000,30266,3,63,Alvisa,1,Oscar Datino,10000,Nafta Super,10000,...,NAFTA SUPER +,3030000,Nafta Super,0.9823,21601.0,3246100.0,855821.0,414800.0,0.9199,Ventas Alvisa
1,2024-01-01 02:54:37.000,29519,3,63,Alvisa,1,Oscar Datino,19000000,Hielo Rolito x 4Kg,2820000,...,Accesorios Hielo,5480000,Hielo,300000.0000,1950000.0,2450200.0,155702.0,398200.0,35500.0000,Ventas Alvisa
2,2024-01-01 02:57:31.000,492880,17,63,Alvisa,1,Oscar Datino,10640000,Bubbaloo Chicle Sab. Vs.,50000,...,Kiosco Caramelos/Chicles/Chup,590000,Kiosco Chicles,30000.0000,0.6,1657000.0,502324.0,201100.0,0.0960,Ventas Alvisa
3,2024-01-01 02:57:31.000,492880,17,63,Alvisa,1,Oscar Datino,10840000,Cabsha Bocadito Chocolate x 10 gr.,50000,...,Kiosco Chocolates,550000,Kiosco Bocaditos,30000.0000,15000.0,1657000.0,502324.0,201100.0,0.2420,Ventas Alvisa
4,2024-01-01 02:57:31.000,492880,17,63,Alvisa,1,Oscar Datino,22100000,Jockey Club Suaves King Size,50000,...,Nobleza Piccardo,1200000,Jockey Club,20000.0000,65000.0,1657000.0,502324.0,201100.0,0.8230,Ventas Alvisa


In [ ]:
#Objetivos
#---------------------------------------------------------------------
#1) Exploración y Limpieza - PIPELINE
def audit_basico(
    df: pd.DataFrame,
    fecha_col: str = "FECHA",
    suc_col: str = "Sucursal",
    montos_cols: List[str] = (
        "VALOR_ARTICULO",
        "COSTO_ARTICULO",
        "VENTA_BRUTA",
        "MONTO_IMPUESTOS_INTERNOS",
        "MONTO_IVA",
        "CANTIDAD_VENDIDA",
    ),
    dup_key_cols: List[str] = (
        "Sucursal",
        "FECHA",
        "NUMERO_TICKET",
        "ID_ARTICULO",
        "CANTIDAD_VENDIDA",
        "VALOR_ARTICULO",
        "COSTO_ARTICULO",
    ),
    fecha_min: str = "2022-01-01",
    fecha_max: str = "2025-12-31",
) -> Dict[str, object]:
    rep: Dict[str, object] = {}

    nfilas = len(df)
    rep["filas_totales"] = nfilas

    # 1) NULOS (por columna)
    nulos = df.isna().sum().sort_values(ascending=False)
    rep["nulos_por_col"] = nulos

    # 2) FORMATO FECHA
    fechas = pd.to_datetime(df[fecha_col], errors="coerce")
    rep["fecha_tipo_ok"] = fechas.dtype.kind == "M" 
    rep["fecha_nat"] = int(fechas.isna().sum())
    rep["fecha_nat_pct"] = 0 if nfilas == 0 else 100 * rep["fecha_nat"] / nfilas
    rep["fecha_min_en_data"] = fechas.min()
    rep["fecha_max_en_data"] = fechas.max()
    fmin, fmax = pd.to_datetime(fecha_min), pd.to_datetime(fecha_max)
    fuera_rango = ((fechas < fmin) | (fechas > fmax)).sum()
    rep["fecha_fuera_rango"] = int(fuera_rango)

    # 2b) FORMATO MONTOS: cuántos NO son numéricos (tras coerción)
    no_numericos = {}
    for col in montos_cols:
        if col in df.columns:
            coerced = pd.to_numeric(df[col], errors="coerce")
            # valores que estaban no nulos pero se volvieron NaN al convertir -> mal formato
            mal = ((df[col].notna()) & (coerced.isna())).sum()
            no_numericos[col] = int(mal)
    rep["montos_no_numericos"] = pd.Series(no_numericos).sort_values(ascending=False) if no_numericos else pd.Series(dtype=int)

    # 3) DUPLICADOS EN UNA MISMA SUCURSAL
    # definimos clave de duplicado (puedes ajustar las columnas clave)
    dup_cols_existentes = [c for c in dup_key_cols if c in df.columns]
    if fecha_col in df.columns and df[fecha_col].dtype.kind != "M":
        # usar fechas parseadas si la original no es datetime
        df__ = df.copy()
        df__["__FECHA_DT"] = fechas
        dup_cols_existentes = [("__FECHA_DT" if c == fecha_col else c) for c in dup_cols_existentes]
    else:
        df__ = df

    if dup_cols_existentes:
        mask_dups = df__.duplicated(subset=dup_cols_existentes, keep=False)
        dups_df = df__.loc[mask_dups].sort_values(dup_cols_existentes)
        # asegurar columna de sucursal
        if suc_col not in dups_df.columns and suc_col in df.columns:
            dups_df[suc_col] = df[suc_col]
        # conteo por sucursal
        if suc_col in dups_df.columns:
            dups_por_suc = dups_df.groupby(suc_col).size().sort_values(ascending=False)
        else:
            dups_por_suc = pd.Series(dtype=int)
        rep["duplicados_total"] = int(mask_dups.sum())
        rep["duplicados_por_sucursal"] = dups_por_suc
        rep["duplicados_detalle"] = dups_df  # (opcional) dataframe con filas duplicadas
    else:
        rep["duplicados_total"] = 0
        rep["duplicados_por_sucursal"] = pd.Series(dtype=int)
        rep["duplicados_detalle"] = pd.DataFrame()

    return rep

reporte = audit_basico(temp)

print("Filas totales:", reporte["filas_totales"])
print("\n--- Nulos por columna (top 15) ---")
print(reporte["nulos_por_col"].head(15))

print("\n--- FECHA ---")
print("¿dtype datetime?:", reporte["fecha_tipo_ok"])
print("NaT:", reporte["fecha_nat"], f"({reporte['fecha_nat_pct']:.2f}%)")
print("Mín/Max en datos:", reporte["fecha_min_en_data"], "/", reporte["fecha_max_en_data"])
print("Fuera de rango esperado:", reporte["fecha_fuera_rango"])

print("\n--- Montos NO numéricos ---")
print(reporte["montos_no_numericos"])

print("\n--- Duplicados ---")
print("Total duplicados:", reporte["duplicados_total"])
print("Duplicados por sucursal (top 10):")
print(reporte["duplicados_por_sucursal"].head(10))

# Si necesitas ver el detalle de filas duplicadas:
reporte["duplicados_detalle"].head(20)




Filas totales: 123718

--- Nulos por columna (top 15) ---
FECHA                       0
RUBRO                       0
Sucursal                    0
COSTO_ARTICULO              0
MONTO_IVA                   0
MONTO_IMPUESTOS_INTERNOS    0
VENTA_BRUTA                 0
VALOR_ARTICULO              0
CANTIDAD_VENDIDA            0
DESC_SUBRUBRO               0
SUBRUBRO                    0
DESC_RUBRO                  0
DESC_DEPARTAMENTO           0
NUMERO_TICKET               0
DEPARTAMENTO                0
dtype: int64

--- FECHA ---
¿dtype datetime?: True
NaT: 0 (0.00%)
Mín/Max en datos: 2024-01-01 00:30:23 / 2024-01-31 23:59:57
Fuera de rango esperado: 0

--- Montos NO numéricos ---
VALOR_ARTICULO              0
COSTO_ARTICULO              0
VENTA_BRUTA                 0
MONTO_IMPUESTOS_INTERNOS    0
MONTO_IVA                   0
CANTIDAD_VENDIDA            0
dtype: int64

--- Duplicados ---
Total duplicados: 0
Duplicados por sucursal (top 10):
Series([], dtype: int64)


,FECHA,NUMERO_TICKET,CANTIDAD_TICKET,ID_SUCURSAL,DESCRIP_SUCURSAL,ID_ZONA_SUPERVISION,DESC_ZONA_SUPERVICION,ID_ARTICULO,DESC_ARTICULO,FAMILIA,...,SUBRUBRO,DESC_SUBRUBRO,CANTIDAD_VENDIDA,VALOR_ARTICULO,VENTA_BRUTA,MONTO_IMPUESTOS_INTERNOS,MONTO_IVA,COSTO_ARTICULO,Sucursal,ID_UNICO


In [83]:
def detectar_outliers(df: pd.DataFrame) -> dict:
    reporte = {}

    # -----------------------------
    # 1. Estadístico (IQR)
    # -----------------------------
    cols_num = ["CANTIDAD_VENDIDA", "VALOR_ARTICULO", "COSTO_ARTICULO", "VENTA_BRUTA"]
    outliers_iqr = {}

    for col in cols_num:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            mask = (df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)
            outliers_iqr[col] = {
                "Q1": Q1, "Q3": Q3, "IQR": IQR,
                "count": int(mask.sum()),
                "pct": 100*mask.sum()/len(df)
            }
    reporte["outliers_iqr"] = pd.DataFrame(outliers_iqr).T

    # -----------------------------
    # 2. Reglas de negocio (ajustadas)
    # -----------------------------
    reglas = {}

    # Cantidad vendida negativa (devoluciones)
    reglas["cant_negativa"] = (df["CANTIDAD_VENDIDA"] < 0).sum()

    # Cantidad cero con venta > 0
    reglas["cant_cero_venta_pos"] = ((df["CANTIDAD_VENDIDA"] == 0) & (df["VENTA_BRUTA"] > 0)).sum()

    # Costos <= 0
    reglas["costo_no_valido"] = (df["COSTO_ARTICULO"] <= 0).sum()

    # Costos mayores al valor
    reglas["costo>valor"] = (df["COSTO_ARTICULO"] > df["VALOR_ARTICULO"]).sum()

    # (❌ Eliminadas venta_inconsistente e iva_fuera_rango)

    reporte["reglas_negocio"] = pd.Series(reglas)

    return reporte


# ==========================
# USO
# ==========================
outlier_report = detectar_outliers(temp)

print("=== OUTLIERS IQR ===")
display(outlier_report["outliers_iqr"])

print("\n=== REGLAS DE NEGOCIO ===")
print(outlier_report["reglas_negocio"])


=== OUTLIERS IQR ===


,Q1,Q3,IQR,count,pct
CANTIDAD_VENDIDA,10000.000,20000.0,10000.000,25550.0,20.651805
VALOR_ARTICULO,21000.000,87500.0,66500.000,20745.0,16.767972
COSTO_ARTICULO,0.573,12650.0,12649.427,8239.0,6.659500
VENTA_BRUTA,749300.000,1891500.0,1142200.000,6893.0,5.571542



=== REGLAS DE NEGOCIO ===
cant_negativa           0
cant_cero_venta_pos     0
costo_no_valido         3
costo>valor            28
dtype: int64


In [ ]:
#Evaluación de resultados - Outliers
# 1) Costos no válidos (<= 0)
costo_no_valido_df = temp[temp["COSTO_ARTICULO"] <= 0]

# 2) Costos mayores al valor
costo_mayor_valor_df = temp[temp["COSTO_ARTICULO"] > temp["VALOR_ARTICULO"]]

print("=== COSTOS NO VÁLIDOS (<=0) ===")
display(costo_no_valido_df[
    ["FECHA","NUMERO_TICKET","Sucursal","ID_ARTICULO","DESC_ARTICULO","CANTIDAD_VENDIDA","VALOR_ARTICULO","COSTO_ARTICULO"]
])

print("\n=== COSTOS > VALOR ===")
display(costo_mayor_valor_df[
    ["FECHA","NUMERO_TICKET","Sucursal","ID_ARTICULO","DESC_ARTICULO","CANTIDAD_VENDIDA","VALOR_ARTICULO","COSTO_ARTICULO"]
])


=== COSTOS NO VÁLIDOS (<=0) ===


,FECHA,NUMERO_TICKET,Sucursal,ID_ARTICULO,DESC_ARTICULO,CANTIDAD_VENDIDA,VALOR_ARTICULO,COSTO_ARTICULO
42411,2024-01-07 15:25:17,21632,Ventas CiudadSF,200000000,Nafta Super Prima,3333483.0,8163700.0,0.0
42413,2024-01-07 15:25:17,21632,Ventas CiudadSF,200040000,Podium - SP Prima,263208.0,776200.0,0.0
42414,2024-01-07 15:26:03,46,Ventas CiudadSF,200030000,G.N.C. Prima,961395.0,1033500.0,0.0



=== COSTOS > VALOR ===


,FECHA,NUMERO_TICKET,Sucursal,ID_ARTICULO,DESC_ARTICULO,CANTIDAD_VENDIDA,VALOR_ARTICULO,COSTO_ARTICULO
13920,2024-01-12 06:43:50,31248,Ventas Alvisa,50000,Podium - SP,0.4548,10001.0000,11244.0000
15285,2024-01-13 06:10:03,418,Ventas Alvisa,50000,Podium - SP,0.4548,10001.0000,11244.0000
23353,2024-01-21 07:59:25,32577,Ventas Alvisa,50000,Podium - SP,0.0318,0.0699,11244.0000
32241,2024-01-29 11:37:47,106516,Ventas Alvisa,30000,Diesel.,0.0395,0.0600,0.9828
33818,2024-01-30 15:40:57,447,Ventas Alvisa,200000000,Nafta Super Prima,0.0950,0.1899,0.7744
39078,2024-01-04 08:23:48,25171,Ventas CiudadSF,40000,G.N.C.,0.0930,0.1000,0.7300
39213,2024-01-04 11:39:30,25179,Ventas CiudadSF,40000,G.N.C.,0.0930,0.1000,0.7300
46915,2024-01-12 06:19:50,25966,Ventas CiudadSF,30000,Diesel.,0.5438,10000.0000,11592.0000
47513,2024-01-12 21:38:16,21998,Ventas CiudadSF,185330000,Lubrax MG4 Mult 20W/40 X 205 L (Suelto),0.2500,24750.0000,48700.0000
55308,2024-01-21 06:49:17,22590,Ventas CiudadSF,10000,Nafta Super,0.0488,0.1000,0.7888


In [ ]:
#2 Conversión de moneda